# Assignment 2

## Abstract


For this assignment we are going to take one of the Kaggle competition and try to apply for our own use, use H2O for the getting the best model and its metrics to see where will our kernel will lie in the leaderboard if we use this and work ahead with the Kaggle competition and further also try to understand how H2O works.

##### Kaggle competition - Kobe Bryant Shot Selection

The Kaggle competition that we have choosen is - Kobe Bryant Shot Selection - Kobe Bryant marked his retirement from the NBA by scoring 60 points in his final game as a Los Angeles Laker on Wednesday, April 12, 2016. Drafted into the NBA at the age of 17, Kobe earned the sport’s highest accolades throughout his long career.

Using 20 years of data on Kobe's swishes and misses, can you predict which shots will find the bottom of the net? This competition is well suited for practicing classification basics, feature engineering, and time series analysis. Practice got Kobe an eight-figure contract and 5 championship rings. What will it get you?

Link to the competition - https://www.kaggle.com/c/kobe-bryant-shot-selection

##### H2O - AutoML

H2O’s AutoML can be used for automating the machine learning workflow, which includes automatic training and tuning of many models within a user-specified time-limit. Stacked Ensembles – one based on all previously trained models, another one on the best model of each family – will be automatically trained on collections of individual models to produce highly predictive ensemble models which, in most cases, will be the top performing models in the AutoML Leaderboard.


We have applied AutoML to our Kobe Bryant Dataset and tried to check how are the models generated by H2O when we specify it the target variable and other predictors with it. Thats the main AIM of this assignment. H2O at the backend(server) try to run various Algorithems that it poses and for various hyperparameter values will generate various models.

Now, the number of models that H2O generates depends on the amount of time you specify it to run for the dataset loaded.

`Colcusion` - We, on the way give certain parameters to H2O to get a good model, with a value of Log Loss(which is the deciding factor of our Kaggle competition) which tuns out to be far too less - `0.34436`. And thus making us win the competition(if we had participated). Thus wesubmit this as an assignment to show, that even we students can win a Kaggle competition.

In [1]:
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split

In [2]:
# Load data and roughly clean it, then sort as game date
df = pd.read_csv("./data.csv")
df.drop(['game_event_id', 'game_id', 'lat', 'lon', 'team_id', 'team_name'], axis=1, inplace=True)
df.sort_values('game_date',  inplace=True)
mask = df['shot_made_flag'].isnull()

In [3]:
# Clean data
actiontypes = dict(df.action_type.value_counts())
df['type'] = df.apply(lambda row: row['action_type'] if actiontypes[row['action_type']] > 20\
                          else row['combined_shot_type'], axis=1)
df.drop(['action_type', 'combined_shot_type'], axis=1, inplace=True)

df['away'] = df.matchup.str.contains('@')
df.drop('matchup', axis=1, inplace=True)

df['distance'] = df.apply(lambda row: row['shot_distance'] if row['shot_distance'] <45 else 45, axis=1)

df['time_remaining'] = df.apply(lambda row: row['minutes_remaining'] * 60 + row['seconds_remaining'], axis=1)
df['last_moments'] = df.apply(lambda row: 1 if row['time_remaining'] < 3 else 0, axis=1)

data = pd.get_dummies(df['type'],prefix="action_type")

features=["away", "period", "playoffs", "shot_type", "shot_zone_area", "shot_zone_basic", "season",
           "shot_zone_range", "opponent", "distance", "minutes_remaining", "last_moments"]
for f in features:
    data = pd.concat([data, pd.get_dummies(df[f], prefix=f),], axis=1)

In [4]:
# Need work on game_date, add this into feature and increse n_estimators can inprove results but waste time and memory 
X = data[~mask]
y = df.shot_made_flag[~mask]

In [5]:
data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25697 entries, 19214 to 19185
Data columns (total 30 columns):
Unnamed: 0                    25697 non-null int64
Unnamed: 0.1                  25697 non-null int64
Unnamed: 0.1.1                25697 non-null int64
Unnamed: 0.1.1.1              25697 non-null int64
Unnamed: 0.1.1.1.1            25697 non-null int64
Unnamed: 0.1.1.1.1.1          25697 non-null int64
Unnamed: 0.1.1.1.1.1.1        25697 non-null int64
Unnamed: 0.1.1.1.1.1.1.1      25697 non-null int64
Unnamed: 0.1.1.1.1.1.1.1.1    25697 non-null int64
loc_x                         25697 non-null int64
loc_y                         25697 non-null int64
minutes_remaining             25697 non-null int64
period                        25697 non-null int64
playoffs                      25697 non-null int64
season                        25697 non-null object
seconds_remaining             25697 non-null int64
shot_distance                 25697 non-null int64
shot_made_flag     

In [39]:
data.corr()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,seconds_remaining,shot_distance,shot_made_flag,team_id,shot_id
Unnamed: 0,1.000000,1.000000,1.000000,0.999997,0.025521,0.760329,-0.033551,-0.012451,0.033551,-0.012451,-0.008314,-0.003366,0.612019,-0.004856,0.020567,-0.013070,NaN,0.999997
Unnamed: 0.1,1.000000,1.000000,1.000000,0.999997,0.025521,0.760329,-0.033551,-0.012451,0.033551,-0.012451,-0.008314,-0.003366,0.612019,-0.004856,0.020567,-0.013070,NaN,0.999997
Unnamed: 0.1.1,1.000000,1.000000,1.000000,0.999997,0.025521,0.760329,-0.033551,-0.012451,0.033551,-0.012451,-0.008314,-0.003366,0.612019,-0.004856,0.020567,-0.013070,NaN,0.999997
Unnamed: 0.1.1.1,0.999997,0.999997,0.999997,1.000000,0.025433,0.761232,-0.033476,-0.012453,0.033476,-0.012453,-0.008251,-0.003357,0.612991,-0.004833,0.020464,-0.013016,NaN,1.000000
game_event_id,0.025521,0.025521,0.025521,0.025433,1.000000,-0.005982,-0.059602,-0.029954,0.059602,-0.029954,-0.274276,0.955914,-0.007963,-0.018670,0.063295,-0.037232,NaN,0.025433
game_id,0.760329,0.760329,0.760329,0.761232,-0.005982,1.000000,0.011361,-0.012944,-0.011361,-0.012944,0.009581,0.005061,0.917898,-0.009029,-0.027247,-0.001612,NaN,0.761232
lat,-0.033551,-0.033551,-0.033551,-0.033476,-0.059602,0.011361,1.000000,0.017578,-1.000000,0.017578,0.077399,-0.039737,-0.000857,0.057766,-0.818124,0.148070,NaN,-0.033476
loc_x,-0.012451,-0.012451,-0.012451,-0.012453,-0.029954,-0.012944,0.017578,1.000000,-0.017578,1.000000,0.006624,-0.030059,-0.007751,0.001512,0.022307,-0.000848,NaN,-0.012453
loc_y,0.033551,0.033551,0.033551,0.033476,0.059602,-0.011361,-1.000000,-0.017578,1.000000,-0.017578,-0.077399,0.039737,0.000857,-0.057766,0.818124,-0.148070,NaN,0.033476
lon,-0.012451,-0.012451,-0.012451,-0.012453,-0.029954,-0.012944,0.017578,1.000000,-0.017578,1.000000,0.006624,-0.030059,-0.007751,0.001512,0.022307,-0.000848,NaN,-0.012453


In [3]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)56-b12)
  Starting server from C:\Users\prabh\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\prabh\AppData\Local\Temp\tmppa1fcuqg
  JVM stdout: C:\Users\prabh\AppData\Local\Temp\tmppa1fcuqg\h2o_prabh_started_from_python.out
  JVM stderr: C:\Users\prabh\AppData\Local\Temp\tmppa1fcuqg\h2o_prabh_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.12.0.1
H2O cluster version age:,"1 year, 8 months and 17 days !!!"
H2O cluster name:,H2O_from_python_prabh_oxqlwl
H2O cluster total nodes:,1
H2O cluster free memory:,3.512 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [4]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("data.csv")
#test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
# Identify predictors and response
x = train.columns
y = "shot_made_flag"
x.remove(y)

In [8]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

In [9]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
# aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=500)
aml = H2OAutoML(max_runtime_secs=1000)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


C:\Users\prabh\Anaconda3\lib\site-packages\h2o\utils\shared_utils.py:162: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss
GBM_grid_0_AutoML_20190224_231345_model_11,0.714417,0.599577
StackedEnsemble_0_AutoML_20190224_231345,0.714221,0.598789
GBM_grid_0_AutoML_20190224_231345_model_6,0.713612,0.60029
GBM_grid_0_AutoML_20190224_231345_model_8,0.712869,0.60245
GBM_grid_0_AutoML_20190224_231345_model_13,0.710736,0.603718
GBM_grid_0_AutoML_20190224_231345_model_12,0.710088,0.602481
GBM_grid_0_AutoML_20190224_231345_model_1,0.708015,0.634247
GLM_grid_0_AutoML_20190224_231345_model_0,0.706425,0.601849
GBM_grid_0_AutoML_20190224_231345_model_7,0.704004,0.611978
GLM_grid_0_AutoML_20190224_231345_model_1,0.701556,0.605131


In [25]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[1])
mod_best.logloss

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_0_AutoML_20190224_231345
No model summary for this model


ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.1764926532356048
RMSE: 0.42011028699093383
LogLoss: 0.5310231582326335
Null degrees of freedom: 17889
Residual degrees of freedom: 17879
Null deviance: 24591.755624125763
Residual deviance: 19000.008601563623
AIC: 19022.008601563623
AUC: 0.8534670498758666
Gini: 0.7069340997517333
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.34965872056497493: 


,0,1,Error,Rate
0,6550.0,3361.0,0.3391,(3361.0/9911.0)
1,1000.0,6979.0,0.1253,(1000.0/7979.0)
Total,7550.0,10340.0,0.2438,(4361.0/17890.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3496587,0.7619412,273.0
max f2,0.3108421,0.8534579,306.0
max f0point5,0.3990243,0.7406377,235.0
max accuracy,0.3717105,0.7683622,255.0
max precision,0.9082062,1.0,0.0
max recall,0.2286628,1.0,368.0
max specificity,0.9082062,1.0,0.0
max absolute_mcc,0.3603428,0.5417507,264.0
max min_per_class_accuracy,0.3797016,0.7658158,249.0
max mean_per_class_accuracy,0.3603428,0.7716452,264.0


Gains/Lift Table: Avg response rate: 44.60 %



...

Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2697246,0.6369760,350.0
max f2,0.2091662,0.8079488,385.0
max f0point5,0.4798416,0.6648308,190.0
max accuracy,0.4798416,0.6849696,190.0
max precision,0.9100023,1.0,0.0
max recall,0.1768559,1.0,392.0
max specificity,0.9100023,1.0,0.0
max absolute_mcc,0.5475715,0.3681010,161.0
max min_per_class_accuracy,0.3728167,0.6412256,260.0
max mean_per_class_accuracy,0.4512397,0.6687256,204.0


Gains/Lift Table: Avg response rate: 45.42 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0101215,0.9011752,2.2016713,2.2016713,1.0,1.0,0.0222841,0.0222841,120.1671309,120.1671309
,2,0.0202429,0.8930345,2.2016713,2.2016713,1.0,1.0,0.0222841,0.0445682,120.1671309,120.1671309
,3,0.0301113,0.8605045,2.0323120,2.1461670,0.9230769,0.9747899,0.0200557,0.0646240,103.2311978,114.6166990
,4,0.0402328,0.8486071,1.9815042,2.1047424,0.9,0.9559748,0.0200557,0.0846797,98.1504178,110.4742384
,5,0.0501012,0.8350341,1.8629526,2.0571171,0.8461538,0.9343434,0.0183844,0.1030641,86.2952646,105.7117132
,6,0.1002024,0.7781616,1.8236065,1.9403618,0.8282828,0.8813131,0.0913649,0.1944290,82.3606539,94.0361836
,7,0.1500506,0.7222752,1.6763995,1.8526711,0.7614213,0.8414840,0.0835655,0.2779944,67.6399474,85.2671135
,8,0.2001518,0.6612739,1.4900200,1.7618937,0.6767677,0.8002528,0.0746518,0.3526462,49.0019977,76.1893728
,9,0.3001012,0.4624755,1.2987074,1.6076285,0.5898734,0.7301855,0.1298050,0.4824513,29.8707380,60.7628460
,10,0.4000506,0.3987628,0.8750947,1.4246108,0.3974684,0.6470588,0.0874652,0.5699164,-12.4905328,42.4610847


<bound method ModelBase.logloss of >

In [26]:
mod_best.model_id

'StackedEnsemble_0_AutoML_20190224_231345'

In [27]:
print(aml.leader.algo)

gbm


In [12]:
# The leader model is stored here
print(aml.leader.logloss)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_0_AutoML_20190224_231345_model_11


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.19715205268336886
RMSE: 0.4440180769781438
LogLoss: 0.5761156622016457
Mean Per-Class Error: 0.3221989214473787
AUC: 0.7545695795221916
Gini: 0.5091391590443832
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.34784643281742506: 


,0,1,Error,Rate
0,5128.0,4783.0,0.4826,(4783.0/9911.0)
1,1590.0,6389.0,0.1993,(1590.0/7979.0)
Total,6718.0,11172.0,0.3562,(6373.0/17890.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3478464,0.6672236,278.0
max f2,0.2697709,0.8090027,342.0
max f0point5,0.6038904,0.6758263,142.0
max accuracy,0.4400834,0.6934600,208.0
max precision,0.9610535,1.0,0.0
max recall,0.1589921,1.0,386.0
max specificity,0.9610535,1.0,0.0
max absolute_mcc,0.6400326,0.3964953,128.0
max min_per_class_accuracy,0.3770550,0.6749067,253.0
max mean_per_class_accuracy,0.3899826,0.6778011,243.0


Gains/Lift Table: Avg response rate: 44.60 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2019-02-24 23:28:10,13 min 24.589 sec,0.0,0.4970758,0.6873045,0.5,1.0,0.5539966,0.4979654,0.6890819,0.5,1.0,0.5457996
,2019-02-24 23:28:10,13 min 24.619 sec,5.0,0.4811590,0.6558964,0.7150690,2.2421356,0.4291224,0.4827778,0.6590497,0.7006591,2.2016713,0.4663462
,2019-02-24 23:28:10,13 min 24.662 sec,10.0,0.4713114,0.6365867,0.7209863,2.2421356,0.4387367,0.4738723,0.6415723,0.6983346,2.2016713,0.5116397
,2019-02-24 23:28:10,13 min 24.727 sec,15.0,0.4646580,0.6231754,0.7240082,2.2421356,0.4449972,0.4679767,0.6296954,0.6993730,2.2016713,0.4483806
,2019-02-24 23:28:10,13 min 24.800 sec,20.0,0.4599724,0.6133952,0.7266182,2.2421356,0.4352711,0.4640170,0.6214440,0.7006980,2.2016713,0.4329453
,2019-02-24 23:28:11,13 min 24.879 sec,25.0,0.4567452,0.6062508,0.7289306,2.2421356,0.4091112,0.4614721,0.6157949,0.7023204,2.2016713,0.4286437
,2019-02-24 23:28:11,13 min 24.973 sec,30.0,0.4541825,0.6004034,0.7314029,2.2421356,0.3995528,0.4600719,0.6124841,0.7019338,2.2016713,0.4380061
,2019-02-24 23:28:11,13 min 25.064 sec,35.0,0.4524496,0.5962480,0.7338780,2.2421356,0.3894913,0.4590322,0.6099674,0.7032252,2.2016713,0.4622976
,2019-02-24 23:28:11,13 min 25.158 sec,40.0,0.4511378,0.5929742,0.7352827,2.2421356,0.4072666,0.4584567,0.6083990,0.7037773,2.2016713,0.4767206
,2019-02-24 23:28:11,13 min 25.261 sec,45.0,0.4498770,0.5898546,0.7388886,2.2421356,0.3819452,0.4580186,0.6071980,0.7050678,2.2016713,0.4552126


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
action_type,5071.3876953,1.0,0.5695870
matchup,1621.9173584,0.3198173,0.1821637
combined_shot_type,660.2830200,0.1301977,0.0741589
shot_distance,484.7258301,0.0955805,0.0544414
opponent,207.9001007,0.0409947,0.0233501
---,---,---,---
C1,4.8532572,0.0009570,0.0005451
Unnamed: 0.1.1,3.5001535,0.0006902,0.0003931
game_id,0.8006709,0.0001579,0.0000899
playoffs,0.0,0.0,0.0



See the whole table with table.as_data_frame()
<bound method ModelBase.logloss of >
